<a href="https://colab.research.google.com/github/rsrohan/Energy-Utilization-of-Rooftops-In-Urban-Areas/blob/datascraping/datascraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
    !mkdir img_mask
except:
    pass

In [0]:
%tensorflow_version 1.x
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from bs4 import BeautifulSoup as bs
import requests
from PIL import Image
import urllib.request

link = 'https://mypages.valdosta.edu/rpmihail/skyfinder/images/index.html'
mainpage = requests.get(link)
soup = bs(mainpage.content,'html.parser')

In [0]:
all_links = soup.find_all('a')


In [0]:
source = 'https://mypages.valdosta.edu/rpmihail/skyfinder/'

for i in range(len(all_links)):
  if all_links[i].get_text('a') == 'mask': 
    print(all_links[i])
    href_mask = all_links[i].get('href')
    href_mask = href_mask.lstrip('../')
    print(href_mask)
    mask_link = source + href_mask
    print(mask_link)
    mask_name = href_mask.lstrip('Masks/')
    print(mask_name)
    with urllib.request.urlopen(mask_link) as url:
      with open('/content/gdrive/My Drive/minor/img_masks/' + mask_name, 'wb') as f:
        f.write(url.read())


In [0]:
import pandas as pd
from PIL import Image
import numpy as np
import os
def rle_encoding(x):
    '''
    x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = np.where(x.T.flatten()==1)[0] # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths


if __name__ == '__main__':
    input_path = '/content/img_mask'
    masks = [f for f in os.listdir(input_path)]
    print(masks)
    masks = sorted(masks)
    print(masks)
    masks.remove('4679.png')
    #masks.remove('.ipynb_checkpoints')
    print(masks)

    encodings = []
    dataset_path = '/content/gdrive/My Drive/minor/dataset'
    img_name = []
    N = len(masks)    # process first N masks
    for i,m in enumerate(masks[:N]):
        if i % 10 == 0: print('{}/{}'.format(i, len(masks)))
        img = Image.open(os.path.join(input_path, m))
        print('image number: {}'.format(masks[i]))
        #print(img)
        x = np.array(img.getdata(), dtype=np.uint8).reshape(img.size[::-1])
        #print(x)
        x = x // 255
        #print(x)
        masks_name = m.rstrip('.png')
        print(masks_name)
        i = 0
        for f in os.listdir(dataset_path):
          if (f.startswith(masks_name)):
            print(f)
            submasks_name = masks_name + '_' + str(i)
            print(submasks_name)
            img_name.append(submasks_name)
            encodings.append(rle_encoding(x))
            i= i+ 1
        #img_name.append()
        
    #check output
    conv_encod = []
    conv = lambda l: ' '.join(map(str, l)) # list -> string
    for i in range(len(encodings)):
      conv_encod.append(conv(encodings[i]))
    #  np.savetxt('masksdata.csv', (masks[i],conv(encodings[i])), delimiter=',')

    mydict = {'img': img_name , 'pixels': conv_encod}
    df = pd.DataFrame(mydict)
    df.to_csv('/content/gdrive/My Drive/minor/dataset/train_masks.csv') 


In [0]:
df_train = pd.read_csv('/content/gdrive/My Drive/minor/train_masks2.csv')
#df_train['img'] = (df_train['img'].astype(str)).rstrip('.png')

df_train.head()

,Unnamed: 0,img,pixels
0,0,10066_0,5061 9 5494 3 5541 10 5973 8 5982 13 6021 10 6...
1,1,10066_1,5061 9 5494 3 5541 10 5973 8 5982 13 6021 10 6...
2,2,10066_2,5061 9 5494 3 5541 10 5973 8 5982 13 6021 10 6...
3,3,10066_3,5061 9 5494 3 5541 10 5973 8 5982 13 6021 10 6...
4,4,10066_4,5061 9 5494 3 5541 10 5973 8 5982 13 6021 10 6...


In [0]:

#@markdown Moving files into sub folders
from google.colab import drive
drive.mount('/content/gdrive')
import os
parent_dir = '/content/gdrive/My Drive/minor/dataset'     
import shutil

for f in os.listdir('/content/gdrive/My Drive/minor/dataset'):
  print(f)
  src = os.path.join('/content/gdrive/My Drive/minor/dataset', f)
  #print(src)
  path = os.path.join(parent_dir, f.split('_')[0])
  try:
   os.mkdir(path)
  except:
    pass    
  dst = os.path.join(path, f)
  print(dst)
  if (f.endswith('.png')):
    shutil.move(src, dst)
